In [1]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  after removing the cwd from sys.path.


In [2]:
df=pd.read_csv('Real_Combine.csv')

In [3]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [4]:
df.dropna(inplace=True)

In [5]:
df[df['PM 2.5'].isna()]
#df['PM 2.5']

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5


In [6]:
X=df.iloc[:,:-1] ## independent features

y=df.iloc[:,-1] ## dependent features



In [7]:
X.head()

,T,TM,Tm,SLP,H,VV,V,VM
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2


## Hyperpatameters
1. How many number of hidden layers we should have ?
2. How many number of neurons we should have in hidden layers ?
3. Learning Rate

In [77]:
#pip install keras-tuner --upgrade

In [8]:
df.isnull().sum()

T         0
TM        0
Tm        0
SLP       0
H         0
VV        0
V         0
VM        0
PM 2.5    0
dtype: int64

In [9]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model



In [15]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='Air Quality Index')



In [16]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)



In [21]:
tuner.search(X_train, y_train,
             epochs=50,
             validation_data=(X_test, y_test))

INFO:tensorflow:Oracle triggered exit


In [22]:
tuner.results_summary()

Results summary
Results in project\Air Quality Index
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 5
units_0: 288
units_1: 256
learning_rate: 0.01
units_2: 32
units_3: 32
units_4: 32
Score: 49.40203603108724
Trial summary
Hyperparameters:
num_layers: 6
units_0: 288
units_1: 192
learning_rate: 0.01
units_2: 384
units_3: 96
units_4: 32
units_5: 128
units_6: 352
units_7: 352
Score: 49.66098658243815
Trial summary
Hyperparameters:
num_layers: 10
units_0: 192
units_1: 256
learning_rate: 0.001
units_2: 352
units_3: 160
units_4: 512
units_5: 416
units_6: 128
units_7: 32
units_8: 32
units_9: 32
Score: 61.58618418375651
Trial summary
Hyperparameters:
num_layers: 20
units_0: 416
units_1: 224
learning_rate: 0.001
units_2: 160
units_3: 384
units_4: 96
units_5: 480
units_6: 256
units_7: 416
units_8: 320
units_9: 512
units_10: 32
units_11: 32
units_12: 32
units_13: 32
units_14: 32
units_15: 32
units_16: 32
units_17: 32
units_18: 32
units_19: 32
Score: 63.40070724487305
Trial summ